# Spaceship Titanic

## Feature Engineering

## Table of Contents
- [Spaceship Titanic](#spaceship-titanic)
- [Feature Engineering](#feature-engineering)
- [Table of Contents](#table-of-contents)
- [Config](#config)
- [Dependencies](#dependencies)
- [Data Extraction](#data-extraction)
- [Dataset Feature Engineering](#dataset-feature-engineering)
    - [Drop Columns](#drop-columns)
    - [Fill Missing Columns](#fill-missing-columns)

### Config

Set up directory variables.

In [11]:
dataset = "spaceship-titanic"
dataset_directory = f"../../datasets/{dataset}"
training_dataset_directory = f"{dataset_directory}/train.csv"
test_dataset_directory = f"{dataset_directory}/test.csv"

### Dependencies

In [12]:
%conda install pandas numpy matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

# All requested packages already installed.




In [13]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.insert(0, '../helpers')

from fill_na_with_mean import fill_na_with_mean

### Data Extraction

In [14]:
spaceship_titanic_train_dataframes = pd.read_csv(training_dataset_directory)
spaceship_titanic_train_dataframes.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [15]:
spaceship_titanic_test_dataframes = pd.read_csv(test_dataset_directory)
spaceship_titanic_test_dataframes.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


#### Dataset Feature Engineering
##### Drop Columns

In [16]:
spaceship_titanic_train_dataframes.drop(['Name'] ,axis=1, inplace=True)
spaceship_titanic_train_dataframes.drop(['Cabin'] ,axis=1, inplace=True)

In [17]:
spaceship_titanic_test_dataframes.drop(['Name'] ,axis=1, inplace=True)
spaceship_titanic_test_dataframes.drop(['Cabin'] ,axis=1, inplace=True)

##### Fill Missing Columns

In [18]:
spaceship_titanic_train_dataframes.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

In [19]:
spaceship_titanic_test_dataframes.isna().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
dtype: int64

In [20]:
training_columns_with_na = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', \
    'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
test_columns_with_na = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', \
    'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

spaceship_titanic_train_dataframes = fill_na_with_mean(dataframe=spaceship_titanic_train_dataframes, columns_to_fill=training_columns_with_na)
spaceship_titanic_test_dataframes = fill_na_with_mean(dataframe=spaceship_titanic_test_dataframes, columns_to_fill=test_columns_with_na)

In [21]:
spaceship_titanic_train_dataframes.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [22]:
spaceship_titanic_test_dataframes.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64